# Laboratorium 1 - content-based recommender

## Przygotowanie

 * pobierz i wypakuj dataset: https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
   * więcej możesz poczytać tutaj: https://grouplens.org/datasets/movielens/
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab1`
 * zainstaluj potrzebne biblioteki:
 `pip install numpy pandas sklearn`

## Część 1. - przygotowanie danych

In [1]:
# importujemy wszystkie potrzebne pakiety

import math
import numpy as np
import pandas

from sklearn.model_selection import train_test_split, KFold

In [2]:
# tworzymy reprezentacje filmow jako wektorow cech - na podstawie gatunkow

genres = [
    '(no genres listed)', 
    'Action', 
    'Adventure', 
    'Animation', 
    'Children', 
    'Comedy', 
    'Crime', 
    'Documentary', 
    'Drama', 
    'Fantasy', 
    'Film-Noir', 
    'Horror', 
    'IMAX', 
    'Musical', 
    'Mystery', 
    'Romance', 
    'Sci-Fi', 
    'Thriller', 
    'War', 
    'Western'
]
genres_no = len(genres)

movies = pandas.read_csv('ml-latest-small/movies.csv')
movies_no = movies.shape[0]

movies['bias'] = 1.0
for genre in genres:
    movies[genre] = np.where(movies['genres'].str.contains(genre, regex=False), 1.0, 0.0)
    
movies = movies.drop(columns=['title', 'genres']).set_index('movieId')
movies

,bias,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193583,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193585,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# wczytujemy oceny uytkownikow i od razu dzielimy je na dwa zbiory - treningowy i testowy

all_ratings = pandas.read_csv('ml-latest-small/ratings.csv').drop(columns=['timestamp'])
train_ratings_set, test_ratings_set = train_test_split(all_ratings, test_size=0.05)
train_ratings_set

,userId,movieId,rating
50837,328,6333,0.5
31380,217,2779,3.0
47592,307,45447,2.0
90300,587,1073,4.0
31011,217,720,4.0
...,...,...,...
73380,474,1090,3.5
69750,448,55765,4.0
13153,84,349,4.0
55731,368,2808,2.0


In [4]:
# inicjalizujemy macierz preferencji uzytkownikow liczbami losowymi z przedzialu [0.0, 5.0]

def initialize_users(raw_ratings):
    users_no = raw_ratings['userId'].unique().size
    users = pandas.DataFrame(5.0 * np.random.uniform(size=(users_no, genres_no+1)), index=raw_ratings['userId'].unique(), columns=['bias']+genres)
    return users_no, users

users_no, users = initialize_users(train_ratings_set)
users

,bias,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
328,1.928594,3.715336,3.246022,1.373340,4.894362,4.688525,3.631357,3.063630,2.948849,2.213758,...,0.586317,0.118343,3.016288,3.539836,0.716375,3.082308,1.870993,4.546908,2.031250,4.474080
217,3.675874,2.629590,1.182415,0.263085,3.491432,1.910120,0.714800,1.907360,2.825630,2.721010,...,4.253374,0.110264,3.610657,2.144217,1.029751,1.483393,1.306323,2.343849,0.259179,1.940810
307,2.181335,0.754096,0.387734,4.865723,1.788651,3.600205,0.288105,0.015700,0.180534,4.620366,...,4.626784,4.704844,1.665424,2.019667,1.778946,3.509620,1.268751,0.756263,0.461027,1.203636
587,2.199712,0.810631,3.241832,4.205916,2.954878,4.933252,3.203908,1.287928,0.133542,4.487915,...,2.732024,0.779223,4.911185,3.627061,4.161955,0.877388,3.823120,2.698626,4.218126,2.175938
232,4.233571,3.259668,0.407624,4.772791,4.072376,3.150514,1.788540,0.902469,0.076524,0.500737,...,1.900583,0.059135,4.847387,4.128055,1.716229,1.224978,1.032791,0.060544,2.595523,3.038712
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,0.218258,1.615999,1.370720,0.011506,0.792652,3.704131,4.253806,0.974801,4.739947,0.679839,...,4.682728,4.451673,0.147901,4.804125,3.221174,2.586804,1.293425,0.344875,0.843394,4.542821
349,2.457987,0.560781,2.116403,4.253111,4.952141,1.663514,4.429848,2.078257,2.284921,3.694557,...,3.328661,2.482384,2.322140,0.320968,2.570446,4.138412,0.526160,2.699083,0.455521,3.069014
231,0.428071,2.186596,0.241234,2.004871,1.958925,3.944566,1.567846,3.618150,3.253661,2.535257,...,4.537302,2.931261,2.419759,0.437041,4.070421,2.951259,4.776319,2.723667,2.316952,0.279333
236,4.679369,3.556160,1.729350,2.336546,4.723732,0.082793,2.040232,3.133864,3.321700,1.050686,...,3.923766,2.734374,4.101570,1.261348,1.621314,4.533727,0.296998,0.285930,4.556304,2.417510


In [5]:
# za pomoca sprytnej sztuczki przeksztalcamy oceny z formatu dostarczonego przez MovieLens do uzytecznej macierzy
# zwroc uwage na to, ze czesci filmow moze brakowac po podziale datasetu na dwie czesci - musimy uzueplnic brakujace kolumny

def get_ratings(raw_ratings, movies):
    ratings = raw_ratings.pivot(*raw_ratings.columns).fillna(0.0)
    missing_movies = set(movies.index).difference(set(raw_ratings['movieId']))
    for movie in missing_movies:
        ratings[movie] = 0.0
    ratings = ratings.reindex(sorted(ratings.columns), axis=1)
    return ratings

ratings = get_ratings(train_ratings_set, movies)
ratings

C:\Users\lenovo\AppData\Local\Temp/ipykernel_9392/4204614947.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  ratings[movie] = 0.0


movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Część 2. - trening modelu

In [89]:
# trenujemy model iteracyjnie, wykorzystujac gradient descent

alpha = 0.0001 # learning speed
delta = 5000 # minimal upgrade for each step
lambd = 0.01 # regularization weight

def calculate_user_preferences(users, movies, ratings, raw_ratings, users_no, movies_no, alpha, delta, lambd):
    total_error = 0.0
    model = users
#     counter = 0

    while(True):
        previous_total_error = total_error

        # mozemy to policzyc jako iloczyn skalarny preferencji uzytkownikow i cech filmow
        predicted_ratings = model.dot(movies.transpose())
#         predicted_ratings = movies.transpose() @ users 
        # tu stosujemy bardzo przydatna funkcje NumPy
        errors = np.where(ratings==0.0, pandas.DataFrame(np.zeros((users_no, movies_no))), predicted_ratings - ratings)
        # znow iloczyn skalarny - tym razem bledow
        gradient = errors.dot(movies)

        # tu stosujemy pewna sztuczke - rozbijamy sobie macierz z wyrazami regularyzujacymi na dwie
#         pierwsza to kolumna zlozona z zer
        regularization_k0 = pandas.DataFrame(np.zeros((users_no, 1)), index=raw_ratings['userId'].unique(), columns=['bias'])
        # druga to macierz preferencji uzytkownikow (czyli modelu) - bez pierwszej kolumny
        regularization_k = model.iloc[:,1:]
        
        # teraz sklejamy obie macierze
        regularization = pandas.concat([regularization_k0, regularization_k], axis=1)

#         # najwazniejszy krok - aktualizacja modelu, czyli wszystkich wag
        model = model - alpha * (gradient + lambd * regularization)

    # suma wszystkich bledow
    
        total_error = errors.sum() # suma elementow macierzy errors
        if math.isnan(total_error):
            break
        print("Total error = " + str(total_error))
        progress = abs(previous_total_error - total_error)
        print("Progress = " + str(progress))
        print("\n\n\n\n")
#         counter += 1
        if progress < delta:
#         if counter > 10:
            break
            
    return model

prediction_model = calculate_user_preferences(users, movies, ratings, train_ratings_set, users_no, movies_no, alpha, delta, lambd)

Total error = 544427.6646965456
Progress = 544427.6646965456





Total error = 530564.1706405137
Progress = 13863.494056031923





Total error = 517005.4843585907
Progress = 13558.686281922972





Total error = 503744.90094072855
Progress = 13260.58341786213





Total error = 490775.8715930645
Progress = 12969.029347664036





Total error = 478091.99844802637
Progress = 12683.873145038146





Total error = 465687.02943797765
Progress = 12404.96901004872





Total error = 453554.8532314807
Progress = 12132.176206496952





Total error = 441689.4942312865
Progress = 11865.359000194177





Total error = 430085.10763318517
Progress = 11604.386598101351





Total error = 418735.9745448532
Progress = 11349.13308833196





Total error = 407636.4971638497
Progress = 11099.477381003497





Total error = 396781.1940139467
Progress = 10855.303149903018





Total error = 386164.69523897505
Progress = 10616.498774971638





Total error = 375781.73795338697
Progress = 10382.95728558808

## Część 3. - ocena jakości algorytmu

In [ ]:
# https://stackoverflow.com/questions/16729574/how-to-get-a-value-from-a-cell-of-a-dataframe

In [58]:
def prepare_for_confusion_matrix(prediction_model, movies, test_ratings_set):
    predicted_ratings = prediction_model.dot(movies.T)
    predicted_ratings = predicted_ratings.astype(int)
    predicted_ratings = predicted_ratings.sort_index(0)

    y_actual = []
    y_predicted = []

    for index, row in test_ratings_set.iterrows():
        y_actual.append(row['rating'])
        y_predicted.append(predicted_ratings.at[int(row['userId']),int(row['movieId'])])

    return y_actual, y_predicted


y_actual, y_predicted = prepare_for_confusion_matrix(prediction_model, movies, test_ratings_set)


C:\Users\lenovo\AppData\Local\Temp/ipykernel_9392/3483175355.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only
  predicted_ratings = predicted_ratings.sort_index(0)


In [ ]:
# https://www.codegrepper.com/code-examples/python/sklearn+knn+example+confusion+matrix

In [ ]:
# https://stackoverflow.com/questions/31324218/scikit-learn-how-to-obtain-true-positive-true-negative-false-positive-and-fal

In [ ]:
# https://stats.stackexchange.com/questions/51296/how-do-you-calculate-precision-and-recall-for-multiclass-classification-using-co

In [76]:
# na podstawie zbioru testowego i wytrenowanego modelu obliczamy metryki opisujace jakosc modelu
## sklearn.confusion_matrix()

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score

positive_threshold = 4.0
negative_threshold = 2.0

# def calculate_stats(test_ratings_set, predicted_ratings, positive_threshold, negative_threshold):
def calculate_stats(y_actual, y_predicted, positive_threshold, negative_threshold):
    # obliczamy true_positives itp.
    # nastepnie wszystkie metryki
    
    y_actual_labels = []
    y_predicted_labels = []
    
    for i in range(0, len(y_actual)):
        if y_actual[i] > int(positive_threshold):
            label = 'Positive'
        elif y_actual[i] <= int(negative_threshold):
            label = 'Negative'
        else:
            label = 'None'
        y_actual_labels.append(label)
    
    for i in range(0, len(y_predicted)):
        if y_predicted[i] > positive_threshold:
            label = 'Positive'
        elif y_predicted[i] < negative_threshold:
            label = 'Negative'
        else:
            label = 'None'
        y_predicted_labels.append(label)
    
    conf_m = confusion_matrix(y_actual_labels, y_predicted_labels)
    
    false_positives = conf_m.sum(axis=0) - np.diag(conf_m)  
    false_negatives = conf_m.sum(axis=1) - np.diag(conf_m)
    true_positives = np.diag(conf_m)
    true_negatives = conf_m.sum() - (false_positives + false_negatives + true_positives)
    
    # fp, ... -> suma elementow w tablicy
    false_positives = false_positives.sum()
    false_negatives = false_negatives.sum()
    true_positives = true_positives.sum()
    true_negatives = true_negatives.sum()

#     print(precision_score(y_actual_labels, y_predicted_labels, average='micro')) # liczy recall, wynik taki sam jak ponizej

        
    recall = true_positives / (true_positives + false_negatives)
    precision = true_positives / (true_positives + false_positives)
    f1 =  2 * ((precision * recall) / (precision + recall))
    accuracy = (true_positives + true_negatives) / (true_positives + false_positives + false_negatives + true_negatives)

    return {
        'true_positives': true_positives,
        'true_negatives': true_negatives,
        'false_positives': false_positives,
        'false_negatives': false_negatives,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [77]:
predicted_ratings = prediction_model.dot(movies.T)
calculate_stats(y_actual, y_predicted, positive_threshold, negative_threshold)

{'true_positives': 838,
 'true_negatives': 5880,
 'false_positives': 4204,
 'false_negatives': 4204,
 'accuracy': 0.44413592489752746,
 'precision': 0.16620388734629116,
 'recall': 0.16620388734629116,
 'f1': 0.16620388734629116}

In [90]:
# dla porownania - obliczmy te same metryki dla modelu losowego
# zauwaz, w jaki sposob ponownie wykorzystujemy funkcje inicjalizujaca preferencje uzytkownikow

# tamten kod opakowac w funkcje zeby zroic tez dla random_prediction

_, random_model = initialize_users(train_ratings_set)
random_prediction = random_model.dot(movies.T)
print(random_prediction)
print("\n\n\n\n")
print(predicted_ratings)







# y_actual, y_predicted = prepare_for_confusion_matrix(prediction_model, movies, test_ratings_set)
y_actual_random, y_predicted_random = prepare_for_confusion_matrix(random_model, movies, train_ratings_set)
calculate_stats(y_actual_random, y_predicted_random, positive_threshold, negative_threshold)

movieId     1          2          3          4         5          6       \
328      21.523673  14.702075   6.471328  10.573476  6.142545  13.582968   
217      16.297760   7.872424  11.913253  14.323090  7.407564   6.307510   
307      13.925578   8.935898   6.927229   8.786441  5.810352  13.263825   
587       8.072730   6.617242   6.616217  10.866168  4.448194   8.664727   
232      12.355733   9.282399   6.877307  10.610423  5.574138  11.970932   
..             ...        ...        ...        ...       ...        ...   
30        9.360783   7.270624   2.701810   3.965745  1.386115   2.045134   
349      18.621310  10.665692  10.664382  13.754558  7.608708  12.984754   
231      14.344701  10.395577   7.279623  12.139993  2.787270  11.602370   
236      15.407194  10.129789   7.440198  10.444861  5.546875  11.882067   
441      18.541774  11.091409   8.051520   8.848147  7.200870   8.960086   

movieId     7          8         9          10      ...     193565     193567  \
328   

C:\Users\lenovo\AppData\Local\Temp/ipykernel_9392/3483175355.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.sort_index will be keyword-only
  predicted_ratings = predicted_ratings.sort_index(0)


{'true_positives': 25561,
 'true_negatives': 121355,
 'false_positives': 70233,
 'false_negatives': 70233,
 'accuracy': 0.51122199720233,
 'precision': 0.266832995803495,
 'recall': 0.266832995803495,
 'f1': 0.266832995803495}

## Część 4. - istotność statystyczna

In [91]:
# wielokrotnie uruchamiamy trening modelu
# za każdym razem dzielimy dataset na zbior treningowy i testowy w inny sposob - klasa KFold robi to za nas
# zwroc uwage na bardzo istotny szczegol - oba modele, wytrenowany i losowy, musza byc porownywane na tym samym zbiorze testowym

n_tests = 5
positive_tests_count = 0
results = []
random_results = []
alpha = 0.01 # learning speed
delta = 500 # minimal upgrade for each step
lambd = 0.1 # regularization weight

# for test, train in KFold(n_splits=n_tests, shuffle=True).split(raw_ratings):
for train, test in KFold(n_splits=n_tests, shuffle=True).split(train_ratings_set):
    # wygeneruj macierz użytkowników i ocen
    # wytrenuj model
    # oblicz metryki dla wytrenowanego modelu
    # oblicz metryki dla modelu losowego
    users_no, users = initialize_users(train_ratings_set)
    ratings = get_ratings(train_ratings_set, movies)
#     ratings = get_ratings(raw_ratings, movies)
    model = calculate_user_preferences(users, movies, ratings, train_ratings_set, users_no, movies_no, alpha, delta, lambd)
    y_actual, y_predicted = prepare_for_confusion_matrix(model, movies, test_ratings_set)
    y_actual_random, y_predicted_random = prepare_for_confusion_matrix(random_model, movies, test_ratings_set)
    stats = calculate_stats(y_actual, y_predicted, positive_threshold, negative_threshold)
    random_stats = calculate_stats(y_actual_random, y_predicted_random, positive_threshold, negative_threshold)
    if stats['recall'] > random_stats['recall']:
        positive_tests_count += 1
    print("Recall for trained: " + str(stats['recall']))
    print("Recall for random: " + str(random_stats['recall']))
    print("\n\n\n\n")
    

C:\Users\lenovo\AppData\Local\Temp/ipykernel_9392/4204614947.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  ratings[movie] = 0.0


Total error = 554255.3529153023
Progress = 554255.3529153023





Total error = -996491.4512241303
Progress = 1550746.8041394325





Total error = 768551.6004387923
Progress = 1765043.0516629226





Total error = -2510246.5271635903
Progress = 3278798.1276023826





Total error = -37592.41490118135
Progress = 2472654.112262409





Total error = -18282284.059816685
Progress = 18244691.644915503





Total error = -102905695.46648301
Progress = 84623411.40666632





Total error = -332131271.744957
Progress = 229225576.27847397





Total error = -780185619.4198035
Progress = 448054347.67484653





Total error = -1433825177.985268
Progress = 653639558.5654646





Total error = -1697703984.2859356
Progress = 263878806.30066752





Total error = 958529871.7765614
Progress = 2656233856.062497





Total error = 13170621728.636307
Progress = 12212091856.859745





Total error = 42649441315.934135
Progress = 29478819587.29783





Total error = 75325364168.95392
Progress = 32675922853

C:\Users\lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\core\_methods.py:48: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
C:\Users\lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\core\_methods.py:48: RuntimeWarning: invalid value encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [92]:
# obliczamy, w ilu probach wytrenowany model okazal sie lepszy od losowego
# przeprowadzamy test statystyczny - jak prawdopodobne jest to, by k pozytywnych prob bylo dzielem przypadku

def possibility_of_at_least_k_successes_in_n(k, n):
    p = 0.0
    # obliczamy kolejno prawdopodobienstwo k sukcesow, k+1 sukcesow, ...
    # przydadza Ci sie funkcje marh.comb() i math.pow()
    for i in range(k, n):
        part = math.comb(n, k)
        part = part / (pow(2, n))
        p += part
    return p

p = 0.05
metric = 'recall'

# positive_tests_count =  # w ilu przypadkach okazalismy sie lepsi niz random?
if possibility_of_at_least_k_successes_in_n(positive_tests_count, n_tests) <= p:
    print('We are better than random!')
else:
    print('There is no evidence we are better')

There is no evidence we are better
